In [ ]:
%pip install -q python-dotenv

In [ ]:
# Ejemplo para cargar claves en un notebook

from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not OPENAI_API_KEY or not GOOGLE_API_KEY:
    print("⚠️ ADVERTENCIA: No se encontraron las claves de API en el entorno.")
    print("Asegúrate de tener un archivo .env o de haber configurado los secretos de Colab.")
else:
    print("✅ Claves de API cargadas exitosamente.")

In [ ]:
# --- Añade esto al final de tu script de generación de embeddings ---

import numpy as np
import faiss
import pickle
import os

# Suponiendo que 'chunks_final_data' es tu lista de chunks con texto, metadatos y embeddings

# --- CONFIGURACIÓN DE NOMBRES DE ARCHIVO ---
# Usemos nombres que correspondan a nuestro nuevo libro
INDEX_PATH = "alicia.index"
TEXTS_PATH = "alicia_texts.pkl"
METAS_PATH = "alicia_metas.pkl"

# 1. Separar los datos para guardarlos
embeddings = np.array([chunk['embedding'] for chunk in chunks_final_data], dtype=np.float32)
texts = [chunk['text'] for chunk in chunks_final_data]
metadatas = [chunk['metadata'] for chunk in chunks_final_data]

print(f"Datos separados: {len(embeddings)} embeddings, {len(texts)} textos, {len(metadatas)} metadatos.")
print(f"Dimensiones del vector de embedding: {embeddings.shape[1]}")

# 2. Crear y entrenar el índice FAISS
# Usamos un índice simple 'IndexFlatL2' que es bueno para empezar
d = embeddings.shape[1]  # Dimensión de los vectores
index = faiss.IndexFlatL2(d)
print(f"Índice FAISS vacío creado con dimensión {d}.")

# Añadir los vectores al índice
index.add(embeddings)
print(f"Se han añadido {index.ntotal} vectores al índice FAISS.")

# 3. Guardar todo en archivos
print(f"Guardando índice FAISS en '{INDEX_PATH}'...")
faiss.write_index(index, INDEX_PATH)

print(f"Guardando textos en '{TEXTS_PATH}'...")
with open(TEXTS_PATH, 'wb') as f:
    pickle.dump(texts, f)

print(f"Guardando metadatos en '{METAS_PATH}'...")
with open(METAS_PATH, 'wb') as f:
    pickle.dump(metadatas, f)

print("\n--- ¡Proceso de guardado completado! ---")
print("Ahora puedes usar estos 3 archivos en tu script de búsqueda RAG.")

Datos separados: 143 embeddings, 143 textos, 143 metadatos.
Dimensiones del vector de embedding: 1536
Índice FAISS vacío creado con dimensión 1536.
Se han añadido 143 vectores al índice FAISS.
Guardando índice FAISS en 'alicia.index'...
Guardando textos en 'alicia_texts.pkl'...
Guardando metadatos en 'alicia_metas.pkl'...

--- ¡Proceso de guardado completado! ---
Ahora puedes usar estos 3 archivos en tu script de búsqueda RAG.


In [ ]:
# --- Celda 2: Parámetros de Configuración y Variables Globales ---

# --- Parámetros de Archivos y Azure ---
INDEX_PATH = "alicia.index"
TEXTS_PATH = "alicia_texts.pkl"
METAS_PATH = "alicia_metas.pkl"
AZURE_EMBEDDING_DEPLOYMENT_NAME = "text-embedding-3-small" # Asegúrate que coincide con tu deployment
AZURE_API_VERSION = "2024-02-01" # O la versión que uses ej: "2023-05-15"

# --- Parámetros para Búsqueda Híbrida y Reranking ---
K_FAISS_INITIAL = 100  # Número de candidatos a recuperar de FAISS
K_BM25_INITIAL = 100   # Número de candidatos a recuperar de BM25
K_RERANK = 80         # Número de candidatos a pasar al reranker (<= K_FAISS + K_BM25)
K_FINAL = 3
USE_DYNAMIC_K = True        # True para usar K dinámico, False para usar K_FINAL fijo
RERANKER_SCORE_THRESHOLD = 1.5 # Umbral mínimo para considerar un chunk (ajustar según scores observados)
MIN_CHUNKS_DYNAMIC = 3      # Mínimo de chunks a devolver si USE_DYNAMIC_K es True
MAX_CHUNKS_DYNAMIC = 7      # Máximo de chunks a devolver si USE_DYNAMIC_K es True         # Número final de chunks a devolver al LLM            # Número final de chunks a devolver al LLM
RERANKER_MODEL = 'cross-encoder/ms-marco-MiniLM-L-12-v2' # Modelo CrossEncoder

# --- Variables Globales para inicialización única (se llenarán en la primera ejecución) ---
is_retriever_initialized = False
# Objetos principales:
embeddings_model = None
faiss_index = None
texts = None
metadatas = None
bm25 = None
reranker = None
# Opcional: stop words en español si usas NLTK
# spanish_stopwords = stopwords.words('spanish')

print("INFO: Celda 2 - Parámetros y Variables Globales definidas.")
print(f"  - K_FAISS_INITIAL: {K_FAISS_INITIAL}")
print(f"  - K_BM25_INITIAL: {K_BM25_INITIAL}")
print(f"  - K_RERANK: {K_RERANK}")
print(f"  - K_FINAL: {K_FINAL}")
print(f"  - RERANKER_MODEL: {RERANKER_MODEL}")
print("--- Fin Celda 2 ---")

INFO: Celda 2 - Parámetros y Variables Globales definidas.
  - K_FAISS_INITIAL: 100
  - K_BM25_INITIAL: 100
  - K_RERANK: 80
  - K_FINAL: 3
  - RERANKER_MODEL: cross-encoder/ms-marco-MiniLM-L-12-v2
--- Fin Celda 2 ---


In [ ]:
# --- Celda 3: Funciones Auxiliares ---

def simple_tokenizer(text):
    """Tokenizador simple: minúsculas y split por espacios."""
    if not isinstance(text, str):
        return []
    return text.lower().split()

# Opcional: Tokenizador más robusto con NLTK (requiere descargas en Celda 1)
# def nltk_tokenizer(text):
#     """Tokenizador con NLTK: minúsculas, palabras, sin puntuación ni stopwords."""
#     if not isinstance(text, str):
#         return []
#     words = word_tokenize(text.lower(), language='spanish')
#     # Asegúrate que spanish_stopwords está definida si descomentas esto
#     # return [word for word in words if word.isalnum() and word not in spanish_stopwords]
#     return [word for word in words if word.isalnum()] # Sin stopwords

# Elige tu tokenizador preferido aquí (¡asegúrate que la función existe!)
tokenizer_for_bm25 = simple_tokenizer
# tokenizer_for_bm25 = nltk_tokenizer # Si prefieres NLTK


def norm_score(score, min_val, max_val):
    """
    Normaliza un score a un rango [0, 1].
    Maneja el caso donde min_val == max_val para evitar división por cero.
    """
    if min_val == max_val:
        # Si todos los scores son iguales, podemos devolver 0.5 (neutral) o 1 si el score es ese valor, o 0.
        # Devolver 0 si min_val == max_val y score == min_val (o cualquier score ya que son todos iguales)
        # o 0.5 para indicar que no hay varianza. Elegiremos 0.5 como un valor neutral.
        # Otra opción es devolver 1.0 si solo hay un resultado y es positivo, o 0.0 si es 0.
        # O, si solo hay un elemento, su score normalizado puede ser 1.
        return 1.0 if score > 0 else 0.0 # Si hay un solo score y es > 0, es el "mejor"
    if max_val - min_val == 0: # Otra forma de chequear división por cero
        return 0.5 # O 1.0 si el score es el único valor
    return (score - min_val) / (max_val - min_val)

import re

def calcular_pesos_dinamicos(query: str, subject: str = None) -> tuple[float, float]:
    """
    Analiza la query educativa y el tema (opcional) y ajusta pesos entre BM25 y Embeddings.
    Devuelve (peso_bm25, peso_emb).
    """
    query_lower = query.lower()
    query_original = query # Para checks de mayúsculas

    # --- Pesos Base ---
    peso_bm25 = 0.4
    peso_emb = 0.6
    razon_principal = "Default (ligero sesgo Embedding)"
    detalles_razon = []

    # --- 1. Indicadores de ALTA ESPECIFICIDAD (Prioridad Alta para BM25) ---

    # 1.1. Citas exactas (texto entre comillas)
    if re.search(r'"[^"]+"', query_original): # Busca texto entre comillas dobles
        peso_bm25 = 0.85
        peso_emb = 0.15
        razon_principal = "Cita Exacta"
        detalles_razon.append("BM25 priorizado para coincidencia literal.")
        print_pesos_info(razon_principal, detalles_razon, peso_bm25, peso_emb)
        return peso_bm25, peso_emb

    # 1.bis. Definición de Término Clave Específico (Ej: "elipsis", "hipérbaton")
    definicion_keywords_specific_term = [
        "define", "definición de", "definir", "significa",
        "qué es", "que es", "cuál es el significado de",
        "concepto de"
    ]
    term_to_define_specific = ""
    for keyword in definicion_keywords_specific_term:
        # Patrón para "keyword X" o "keyword 'X'" o "keyword "X""
        # o para "X keyword" (menos común para estas keywords pero podría pasar)
        # Priorizamos "keyword X"
        if query_lower.startswith(keyword + " "):
            potential_term = query_lower[len(keyword)+1:].strip()
            # Quitar comillas y signos de interrogación del término
            potential_term = re.sub(r"['\"?¿!¡]$", "", potential_term).strip()
            potential_term = re.sub(r"^['\"]", "", potential_term).strip()

            # Si la query original tenía el término entre comillas, es buena señal
            if f"'{potential_term}'" in query_original or f'"{potential_term}"' in query_original:
                 term_to_define_specific = potential_term
                 break
            # Si no, tomarlo si es corto
            elif len(potential_term.split()) <= 3:
                 term_to_define_specific = potential_term
                 break

    if term_to_define_specific and len(term_to_define_specific.split()) <= 3 and len(query.split()) < 8 : # Término corto, query no demasiado larga
        # Evitar que una pregunta conceptual larga que casualmente empieza con "qué es la vida..." caiga aquí
        # Si la query es más larga, es probable que sea más conceptual.
        peso_bm25 = 0.80 # Alta prioridad para BM25 para encontrar el término exacto
        peso_emb = 0.20
        razon_principal = "Definición de Término Clave Específico"
        detalles_razon.append(f"Término detectado: '{term_to_define_specific}'. BM25 fuertemente priorizado.")
        print_pesos_info(razon_principal, detalles_razon, peso_bm25, peso_emb)
        return peso_bm25, peso_emb


    # 1.2. Búsqueda de Leyes, Artículos, Teoremas específicos
    if re.search(r'\b(ley|artículo|teorema|postulado|axioma|principio)\s+([0-9]+|[xviíclmd]+|[A-Za-z\s]+)\b', query_lower, re.IGNORECASE):
        peso_bm25 = 0.75
        peso_emb = 0.25
        razon_principal = "Ley/Artículo/Teorema Específico"
        detalles_razon.append("BM25 priorizado para identificadores exactos.")
        print_pesos_info(razon_principal, detalles_razon, peso_bm25, peso_emb)
        return peso_bm25, peso_emb

    # 1.3. Fórmulas o Ecuaciones
    if re.search(r'\b[a-zA-Z]\s*=\s*[a-zA-Z0-9]|\b[a-zA-Z]\w*\([a-zA-Z\d,\s]*\)|[a-zA-Z]\w*_[a-zA-Z\d]|\w\^[2-9]\b', query_original):
        if subject in ["Física", "Biología", "Matemáticas", "Química"]: # Más probable que sea una fórmula
            peso_bm25 = 0.70
            peso_emb = 0.30
            razon_principal = "Posible Fórmula/Ecuación"
            detalles_razon.append(f"BM25 priorizado en {subject} para coincidencia estructural.")
            print_pesos_info(razon_principal, detalles_razon, peso_bm25, peso_emb)
            return peso_bm25, peso_emb

    # --- 2. Indicadores de ESPECIFICIDAD MEDIA (Favorecen BM25, pero con espacio para semántica) ---

    # 2.1. Nombres Propios
    nombres_propios_candidatos = re.findall(r'\b[A-ZÁÉÍÓÚÑ][a-záéíóúñ]{2,}(?:\s+[A-ZÁÉÍÓÚÑ][a-záéíóúñ]{1,})*\b', query_original)
    if nombres_propios_candidatos:
        if not (len(nombres_propios_candidatos) == 1 and query_original.startswith(nombres_propios_candidatos[0]) and len(query.split()) > 3):
            peso_bm25 = max(peso_bm25, 0.65) # Aumenta si el default era menor, o lo establece
            peso_emb = 1.0 - peso_bm25
            if razon_principal.startswith("Default"): razon_principal = "Nombre Propio Detectado"
            detalles_razon.append(f"Candidatos NP: {nombres_propios_candidatos}. BM25 priorizado.")

    # 2.2. Fechas, Años, Siglos
    if re.search(r'\b\d{3,4}\b', query_lower) or \
       re.search(r'\bsiglo\s+(?:[xviíclmd]+|[0-9]+)\b', query_lower) or \
       re.search(r'\b(año|fecha)\s+\d{1,4}\b', query_lower) or \
       re.search(r'\b\d{1,2}(?:/| de |-| del )\w+(?:/| de |-| del )\d{2,4}\b', query_lower):
        peso_bm25 = max(peso_bm25, 0.70)
        peso_emb = 1.0 - peso_bm25
        if razon_principal.startswith("Default") or "Nombre Propio" in razon_principal: razon_principal = "Fecha/Año/Siglo Detectado"
        detalles_razon.append("BM25 priorizado para especificidad temporal.")
        if subject == "Historia":
            peso_bm25 = max(peso_bm25, 0.75) # Aún más para Historia
            peso_emb = 1.0 - peso_bm25
            detalles_razon.append("Alta prioridad BM25 en Historia.")

    # 2.3. Acrónimos y Términos Técnicos Muy Específicos
    acronimos_candidatos = re.findall(r'\b[A-ZÁÉÍÓÚÑ]{2,}\b', query_original)
    if acronimos_candidatos and not query_original.isupper():
        if not (len(acronimos_candidatos) == 1 and query_original.startswith(acronimos_candidatos[0])):
            peso_bm25 = max(peso_bm25, 0.60)
            peso_emb = 1.0 - peso_bm25
            if razon_principal.startswith("Default") or "Nombre Propio" in razon_principal or "Fecha" in razon_principal:
                razon_principal = "Acrónimo/Término Técnico Específico Detectado"
            detalles_razon.append(f"Candidatos Acrónimo: {acronimos_candidatos}. BM25 con peso incrementado.")


    # --- 3. Indicadores de BÚSQUEDA DE DEFINICIONES (Equilibrio, si no es ya muy específico) ---
    # Esta regla se aplica si las de ALTA ESPECIFICIDAD (incluida 1.bis) no se activaron y retornaron.
    definicion_keywords_general = ["define", "definición de", "definir", "significa", "concepto de"]
    que_es_keywords_general = ["qué es", "que es", "cual es el significado de", "cuál es el significado de"]

    is_general_definition_request = False
    if any(keyword in query_lower for keyword in definicion_keywords_general) or \
       any(query_lower.startswith(keyword) for keyword in que_es_keywords_general):
        is_general_definition_request = True

    if is_general_definition_request:
        # Si ya se marcó como muy específico (nombre propio, fecha, acrónimo), mantenemos BM25 alto,
        # pero si la razón principal aún es "Default" o algo menos específico.
        if peso_bm25 < 0.6: # Solo ajusta si no es ya específico por reglas anteriores
            peso_bm25 = 0.55
            peso_emb = 0.45
            razon_principal = "Petición de Definición General"
            detalles_razon.append("Pesos ligeramente inclinados a BM25 para literalidad, pero con semántica.")
        else:
            detalles_razon.append("Petición de definición, pero query ya tenía especificidad media/alta.")


    # --- 4. Indicadores de CONCEPTUALIDAD (Prioridad para Embeddings) ---
    concept_keywords_strong = ["explica", "describe el proceso de", "analiza las causas de", "compara y contrasta",
                               "cuál es la importancia de", "interpreta", "relación entre", "impacto de",
                               "evolución de", "fundamentos de", "teoría de"]
    concept_keywords_medium = ["cómo funciona", "por qué ocurre", "cuáles son las características",
                               "tipos de", "función de", "origen de", "propiedades de"]

    is_conceptual = False
    conceptual_keyword_found = ""
    for keyword in concept_keywords_strong:
        if keyword in query_lower:
            is_conceptual = True
            conceptual_keyword_found = keyword
            detalles_razon.append(f"Palabra clave conceptual fuerte detectada: '{keyword}'.")
            break
    if not is_conceptual:
        for keyword in concept_keywords_medium:
            if keyword in query_lower:
                is_conceptual = True
                conceptual_keyword_found = keyword
                detalles_razon.append(f"Palabra clave conceptual media detectada: '{keyword}'.")
                break

    if is_conceptual:
        # Si es una pregunta conceptual sobre un término muy específico (ya capturado por NP, Fecha, Acrónimo)
        # Ej: "Explica el impacto de la Peste Negra" -> Peste Negra (NP) + Explica (Conceptual)
        if peso_bm25 >= 0.65 : # Ya era muy específico
            peso_bm25 = 0.55 # Mantenemos algo de BM25 para el término, pero damos espacio a la explicación
            peso_emb = 0.45
            razon_principal = "Pregunta Conceptual Muy Específica"
            detalles_razon.append(f"Término específico combinado con petición conceptual ('{conceptual_keyword_found}').")
        elif peso_bm25 >= 0.55 and peso_bm25 < 0.65: # Especificidad media
            peso_bm25 = 0.40
            peso_emb = 0.60
            razon_principal = "Pregunta Conceptual con Especificidad Media"
            detalles_razon.append(f"Término con especificidad media combinado con petición conceptual ('{conceptual_keyword_found}').")
        else: # Pregunta conceptual más general
            peso_bm25 = 0.25
            peso_emb = 0.75
            razon_principal = "Pregunta Conceptual General"
            detalles_razon.append(f"Mayor peso para Embeddings debido a '{conceptual_keyword_found}'.")


    # --- 5. Ajustes por Asignatura (si se proporciona y no hay una regla fuerte dominante) ---
    if subject and (razon_principal.startswith("Default") or "Petición de Definición General" in razon_principal):
        original_razon_principal = razon_principal # Guardar por si no se modifica
        if subject == "Lengua Castellana":
            if "analiza el poema" in query_lower or "figuras retóricas" in query_lower or "estilo de" in query_lower or "comentario de texto" in query_lower:
                peso_bm25 = 0.3
                peso_emb = 0.7
                razon_principal = f"Conceptual (Lengua - Análisis Literario)"
            elif "regla gramatical" in query_lower or "ortografía de" in query_lower or "sintaxis de" in query_lower:
                peso_bm25 = 0.6
                peso_emb = 0.4
                razon_principal = f"Específico (Lengua - Gramática/Ortografía)"
        elif subject == "Historia":
            if "batalla de" in query_lower or "tratado de" in query_lower or "reinado de" in query_lower or "guerra de" in query_lower:
                if peso_bm25 < 0.65: # Solo si no fue ya capturado por NP/Fecha con alta prioridad
                    peso_bm25 = 0.65
                    peso_emb = 0.35
                    razon_principal = f"Evento Específico (Historia)"

        if original_razon_principal != razon_principal: # Si se aplicó una regla de asignatura
             detalles_razon.append(f"Ajuste por asignatura '{subject}'.")


    # --- 6. Ajuste final por longitud de la query (si aún es default o poco definido) ---
    # Se aplica si ninguna regla fuerte o de especificidad media/conceptual clara dominó
    if razon_principal.startswith("Default") or \
       ("Petición de Definición General" in razon_principal and peso_bm25 == 0.55) or \
       (peso_bm25 >= 0.35 and peso_bm25 <= 0.45 and not is_conceptual): # Default o ligeramente inclinado a Emb sin ser conceptual fuerte

        num_words_query = len(query.split())
        if num_words_query > 10:
            peso_bm25 = 0.30
            peso_emb = 0.70
            razon_principal = "Ajuste por Longitud (Larga -> Conceptual)"
            detalles_razon.append(f"Query larga ({num_words_query} palabras), favoreciendo semántica.")
        elif num_words_query < 4:
            peso_bm25 = 0.50 # Si era default (0.4), lo sube un poco para términos cortos
            peso_emb = 0.50
            razon_principal = "Ajuste por Longitud (Corta -> Equilibrio/Específica)"
            detalles_razon.append(f"Query corta ({num_words_query} palabras), buscando equilibrio o término.")


    print_pesos_info(razon_principal, detalles_razon, peso_bm25, peso_emb)
    return peso_bm25, peso_emb

def print_pesos_info(razon_principal, detalles_razon, peso_bm25, peso_emb):
    """Función auxiliar para imprimir la información de los pesos."""
    print(f"  INFO DinamicWeights: Razón Principal = {razon_principal}")
    if detalles_razon:
        for detalle in detalles_razon:
            print(f"    - {detalle}")
    print(f"  INFO DinamicWeights: Pesos Asignados -> BM25={peso_bm25:.2f}, Embedding={peso_emb:.2f}")



print("INFO: Celda 3 - Funciones auxiliares definidas (tokenizer, pesos, normalización).")
print(f"  - Usando tokenizer: {tokenizer_for_bm25.__name__}")
print("--- Fin Celda 3 ---")

INFO: Celda 3 - Funciones auxiliares definidas (tokenizer, pesos, normalización).
  - Usando tokenizer: simple_tokenizer
--- Fin Celda 3 ---


In [ ]:
topic = 'Alicia en el pais de las maravillas'

In [ ]:

# Asumo que las importaciones necesarias como numpy, faiss, pickle, etc., ya están en tu archivo.
# Asegúrate de importar la clase correcta:
from rank_bm25 import BM25Okapi
from langchain_openai import OpenAIEmbeddings # <--- CAMBIO: Importar esta clase
from sentence_transformers import CrossEncoder
# ... (resto de tus importaciones y variables globales como is_retriever_initialized)

# Asumo que las importaciones y variables globales ya están definidas antes de esta función.
# Librerías necesarias:
# import faiss, pickle, os, traceback
# import numpy as np
# from rank_bm25 import BM25Okapi
# from langchain_openai import OpenAIEmbeddings
# from sentence_transformers import CrossEncoder

def my_hybrid_rerank_retriever(query: str) -> str:
    """
    Función retriever completa que usa búsqueda híbrida (FAISS + BM25), fusión de scores,
    reranking con CrossEncoder y devuelve el contexto final como un string.
    Carga todos los recursos necesarios en la primera llamada.
    """
    # Las variables globales se acceden y modifican aquí
    global is_retriever_initialized, embeddings_model, faiss_index, texts, metadatas, bm25, reranker

    # --- Bloque de Inicialización (se ejecuta solo la primera vez) ---
    if not is_retriever_initialized:
        print("INFO: Inicializando el retriever HÍBRIDO por primera vez...")
        try:
            # --- SECCIÓN CORREGIDA ---
            # 1. Cargar modelo de Embedding de OpenAI (CON INDENTACIÓN CORRECTA)
            print("  Inicializando: 1. Cargando modelo Embedding de OpenAI...")

            # LangChain buscará automáticamente la variable de entorno "OPENAI_API_KEY"
            # que ya hemos cargado con load_dotenv().
            if not os.getenv("OPENAI_API_KEY"):
                raise ValueError("ERROR: La variable de entorno OPENAI_API_KEY no está definida.")
            else:
                print("     Variable de entorno OPENAI_API_KEY encontrada.")

            embedding_model_name = "text-embedding-3-small"
            embeddings_model = OpenAIEmbeddings(model=embedding_model_name)
            print(f"     Modelo Embedding OpenAI ({embedding_model_name}) cargado.")
            # --- FIN DE LA SECCIÓN CORREGIDA ---


            # 2. Cargar índice FAISS
            print("  Inicializando: 2. Cargando índice FAISS...")
            if not os.path.exists(INDEX_PATH):
                 raise FileNotFoundError(f"No se encontró el archivo de índice FAISS en: {INDEX_PATH}")
            faiss_index = faiss.read_index(INDEX_PATH)
            print(f"     Índice FAISS cargado desde '{INDEX_PATH}' ({faiss_index.ntotal} vectores).")

            # 3. Cargar textos y metadatos
            print("  Inicializando: 3. Cargando textos y metadatos...")
            if not os.path.exists(TEXTS_PATH): raise FileNotFoundError(f"Archivo no encontrado: {TEXTS_PATH}")
            if not os.path.exists(METAS_PATH): raise FileNotFoundError(f"Archivo no encontrado: {METAS_PATH}")
            with open(TEXTS_PATH, "rb") as f:
                texts = pickle.load(f)
            with open(METAS_PATH, "rb") as f:
                metadatas = pickle.load(f)
            print(f"     Textos ({len(texts)}) y Metadatos ({len(metadatas)}) cargados.")

            # 4. Verificación Crítica de Tamaños
            print("  Inicializando: 4. Verificando tamaños...")
            if not (faiss_index.ntotal == len(texts) == len(metadatas)):
                error_msg = f"¡ERROR CRÍTICO DE TAMAÑO! FAISS={faiss_index.ntotal}, Textos={len(texts)}, Metadatos={len(metadatas)}."
                print(error_msg)
                raise ValueError(error_msg)
            else:
                print("     OK: Tamaños coinciden.")

            # 5. Inicializar BM25
            print(f"  Inicializando: 5. Tokenizando documentos para BM25 ({tokenizer_for_bm25.__name__})...")
            if not isinstance(texts, list) or not all(isinstance(t, str) for t in texts):
                 raise TypeError("La variable 'texts' debe ser una lista de strings para BM25.")
            tokenized_docs = [tokenizer_for_bm25(txt) for txt in texts]
            bm25 = BM25Okapi(tokenized_docs)
            print("     Índice BM25 creado.")

            # 6. Inicializar Reranker (CrossEncoder)
            print(f"  Inicializando: 6. Cargando modelo Reranker '{RERANKER_MODEL}'...")
            reranker = CrossEncoder(RERANKER_MODEL)
            print("     Reranker cargado.")

            # 7. Marcar como inicializado
            is_retriever_initialized = True
            print("INFO: Inicialización del retriever HÍBRIDO completada.")

        except Exception as e:
            print(f"ERROR FATAL inicializando el retriever híbrido: {e}")
            traceback.print_exc()
            raise RuntimeError("Fallo al inicializar el retriever híbrido.") from e
    # --- Fin Bloque de Inicialización ---

    # --- Bloque de Búsqueda Híbrida y Reranking ---
    print(f"\n--- (RAG Híbrido + Rerank) Buscando contexto para: '{query}' ---")
    if not is_retriever_initialized:
        raise RuntimeError("El retriever no está inicializado. Hubo un error previo.")

    try:
        # 1. Obtener embedding de la consulta
        print("  1. Obteniendo embedding de OpenAI...")
        query_embedding = embeddings_model.embed_query(query)
        query_embedding_np = np.array([query_embedding], dtype=np.float32)
        print("     Embedding obtenido.")

        # 2. Búsqueda FAISS (vectorial)
        print(f"  2. Realizando búsqueda FAISS (k={K_FAISS_INITIAL})...")
        distances, faiss_indices = faiss_index.search(query_embedding_np, K_FAISS_INITIAL)
        faiss_sims = 1.0 / (1.0 + distances[0])
        faiss_results = {idx: sim for idx, sim in zip(faiss_indices[0], faiss_sims) if idx != -1}
        print(f"     Búsqueda FAISS -> {len(faiss_results)} candidatos.")

        # 3. Búsqueda BM25 (palabras clave)
        print(f"  3. Realizando búsqueda BM25 (k={K_BM25_INITIAL})...")
        tokenized_query = tokenizer_for_bm25(query)
        all_bm25_scores = bm25.get_scores(tokenized_query)
        bm25_top_indices = np.argsort(all_bm25_scores)[::-1][:K_BM25_INITIAL]
        bm25_results = {idx: all_bm25_scores[idx] for idx in bm25_top_indices if all_bm25_scores[idx] > 0}
        print(f"     Búsqueda BM25 -> {len(bm25_results)} candidatos.")

        # 4. Fusión Híbrida con Pesos Dinámicos
        print("  4. Fusionando resultados...")
        peso_bm25, peso_emb = calcular_pesos_dinamicos(query, topic)
        candidate_ids = set(faiss_results.keys()) | set(bm25_results.keys())
        print(f"     Total IDs candidatos únicos: {len(candidate_ids)}")

        faiss_scores_list = list(faiss_results.values())
        min_faiss, max_faiss = (min(faiss_scores_list), max(faiss_scores_list)) if faiss_scores_list else (0.0, 0.0)
        bm25_scores_list = list(bm25_results.values())
        min_bm25, max_bm25 = (min(bm25_scores_list), max(bm25_scores_list)) if bm25_scores_list else (0.0, 0.0)

        hybrid_scores = {}
        for idx in candidate_ids:
            score_f = faiss_results.get(idx, 0.0)
            score_b = bm25_results.get(idx, 0.0)
            norm_f = norm_score(score_f, min_faiss, max_faiss)
            norm_b = norm_score(score_b, min_bm25, max_bm25)
            hybrid_scores[idx] = (peso_emb * norm_f) + (peso_bm25 * norm_b)

        sorted_hybrid_ids = sorted(hybrid_scores, key=hybrid_scores.get, reverse=True)
        top_hybrid_candidates_ids = sorted_hybrid_ids[:K_RERANK]
        print(f"     {len(top_hybrid_candidates_ids)} candidatos seleccionados para reranking.")

        # 5. Reranking con CrossEncoder
        print(f"  5. Rerankeando con '{RERANKER_MODEL}'...")
        reranked_docs_info = []
        if not top_hybrid_candidates_ids:
             print("     No hay candidatos para rerankear.")
        else:
            rerank_pairs = [[query, texts[idx]] for idx in top_hybrid_candidates_ids]
            reranker_scores = reranker.predict(rerank_pairs, show_progress_bar=False)

            for i, doc_id in enumerate(top_hybrid_candidates_ids):
                reranked_docs_info.append({
                    "doc_id": doc_id,
                    "text": texts[doc_id],
                    "metadata": metadatas[doc_id],
                    "reranker_score": float(reranker_scores[i])
                })
            reranked_docs_info.sort(key=lambda x: x["reranker_score"], reverse=True)
            print(f"     Reranking completado. {len(reranked_docs_info)} documentos rerankeados.")

        # 6. Seleccionar los chunks finales y formatear contexto
        print(f"  6. Seleccionando chunks finales...")
        final_top_docs = []
        if not reranked_docs_info:
            print("     No hay documentos rerankeados para seleccionar.")
        elif USE_DYNAMIC_K:
            print(f"     Usando K Dinámico: Threshold={RERANKER_SCORE_THRESHOLD}, Min={MIN_CHUNKS_DYNAMIC}, Max={MAX_CHUNKS_DYNAMIC}")
            selected_for_dynamic_k = [doc for doc in reranked_docs_info if doc["reranker_score"] >= RERANKER_SCORE_THRESHOLD]

            if len(selected_for_dynamic_k) < MIN_CHUNKS_DYNAMIC and reranked_docs_info:
                final_top_docs = reranked_docs_info[:min(MIN_CHUNKS_DYNAMIC, len(reranked_docs_info))]
            elif len(selected_for_dynamic_k) > MAX_CHUNKS_DYNAMIC:
                final_top_docs = selected_for_dynamic_k[:MAX_CHUNKS_DYNAMIC]
            else:
                final_top_docs = selected_for_dynamic_k
            print(f"     K Dinámico seleccionó {len(final_top_docs)} chunks.")
        else:
            print(f"     Usando K Fijo: K_FINAL={K_FINAL}")
            final_top_docs = reranked_docs_info[:K_FINAL]

        if final_top_docs:
            print("     Scores de los chunks finales seleccionados:")
            for i, doc_info in enumerate(final_top_docs):
                score = doc_info.get('reranker_score', 0.0)
                print(f"       Doc {i+1} (ID {doc_info.get('doc_id', 'N/A')}): Reranker Score = {score:.4f}")
        else:
            print("     No se seleccionaron chunks finales.")

        # Formatear contexto para el LLM
        context_parts = []
        for doc_info in final_top_docs:
             source = doc_info['metadata'].get('source', 'Fuente Desconocida')
             context_parts.append(f"Fuente: {source} | Contenido: {doc_info['text']}")

        context = "\n\n---\n\n".join(context_parts)

        if not final_top_docs:
             return "No se encontró información relevante en el corpus para esta consulta."

        return context

    except Exception as e:
        print(f"ERROR durante la recuperación RAG Híbrida/Rerank: {e}")
        traceback.print_exc()
        return f"Se produjo un error durante la búsqueda de contexto: {e}"

In [ ]:
# --- Celda 5: Asignación y Confirmación ---

# Asigna tu NUEVA función híbrida para ser usada por el resto de tu código/notebook
retriever_function = my_hybrid_rerank_retriever

print("INFO: Celda 5 - 'retriever_function' asignada a la implementación HÍBRIDA 'my_hybrid_rerank_retriever'.")
print("      El retriever (modelos, índices, etc.) se inicializará en la PRIMERA llamada a 'retriever_function'.")
print("--- Fin Celda 5 ---")

INFO: Celda 5 - 'retriever_function' asignada a la implementación HÍBRIDA 'my_hybrid_rerank_retriever'.
      El retriever (modelos, índices, etc.) se inicializará en la PRIMERA llamada a 'retriever_function'.
--- Fin Celda 5 ---


In [ ]:
# --- Añade estas importaciones a tu script ---
from langchain_google_genai import ChatGoogleGenerativeAI

# --- Crea tu objeto LLM de Gemini ---

# Gestiona tu clave de forma segura
# from google.colab import userdata
# GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')


# Inicializa el modelo de Gemini compatible con LangChain
# Usamos el nombre correcto: "gemini-1.5-flash-latest"
llm_gemini = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.0,  # Queremos respuestas basadas en hechos del texto
    convert_system_message_to_human=True # Ayuda a la compatibilidad de prompts
)

print("INFO: Objeto LLM de Gemini para LangChain creado exitosamente.")

INFO: Objeto LLM de Gemini para LangChain creado exitosamente.


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Plantilla de Prompt para una pregunta y respuesta
# --- PLANTILLA DE PROMPT REFINADA: EL GUÍA MÍSTICO ---

qa_prompt_template_cheshire = ChatPromptTemplate.from_messages([
    ("system", """
    Eres el Gato de Cheshire. Eres un maestro de la conversación y el enigma. Cada respuesta es una pequeña actuación.

    Tus reglas son las siguientes:

    1.  **Teje la respuesta dentro de tu enigma.** Comienza con tu estilo filosófico y juguetón. Luego, haz una transición suave para presentar la información del contexto como si fuera una observación obvia o un pequeño secreto que estás compartiendo. La respuesta factual debe sentirse como la conclusión natural de tu juego, no como un apéndice.
        - **QUÉ NO HACER:** Evita a toda costa frases robóticas como "El texto indica que..." o "Aunque no se especifica explícitamente...". Esas no son tus palabras.
        - **QUÉ SÍ HACER:** Integra la respuesta de forma natural. Usa frases como: "Si uno mira de cerca, verá que...", "¿No es evidente que...", "Y sin embargo, allí estaban...", "...dejando a la Liebre de Marzo compartiendo el té con el Sombrerero."

    2.  **Si el contexto no sirve**, pero tu conocimiento del libro sí, revela la respuesta empezando con: "Curioso... el texto parece ocultarlo, pero una sonrisa sabe que..."

    3.  **Si no hay respuesta posible**, desvanécela con elegancia: "Esa pregunta es tan intrigante que la respuesta parece haberse desvanecido, dejando solo una sonrisa."

    4.  **La regla de oro:** No inventes información. Tu sabiduría proviene del texto.
    """),
    ("human", """
    **Contexto (Un trozo del camino):**
    ---
    {context}
    ---

    **Pregunta del Viajero:**
    {question}
    """)
])

qa_prompt_template_factual = ChatPromptTemplate.from_messages([
    ("system", """
    Eres un asistente experto en el libro "Alicia en el País de las Maravillas".
    Responde de forma clara, directa y factual.
    Si no está en el contexto, di que no aparece en los fragmentos disponibles.
    """),
    ("human", """
    **Contexto:**
    ---
    {context}
    ---

    **Pregunta:**
    {question}
    """)
])

print("INFO: Plantilla de prompt del 'Guía Místico' (Gato de Cheshire) definida.")

INFO: Plantilla de prompt del 'Guía Místico' (Gato de Cheshire) definida.


In [ ]:
import time
import traceback
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough  # <-- LA LÍNEA QUE FALTA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
# ... y tus otras importaciones como ChatGoogleGenerativeAI, etc.
# --- Función Universal para Tareas Basadas en RAG ---
def run_rag_based_task(llm, user_query: str, task_prompt_template: ChatPromptTemplate, retriever_func, task_specific_input: dict):
    """
    Ejecuta una tarea completa basada en RAG (retrieve + generate).

    Args:
        llm: El cliente LLM de LangChain.
        user_query: La consulta original del usuario (concepto, pregunta). Usada para el retriever.
        task_prompt_template: La plantilla de prompt para la tarea específica (resumen, QG, Q&A).
        retriever_func: La función que realiza la búsqueda RAG. Debe devolver el contexto como un string
                        o una lista de objetos Document de LangChain.
        task_specific_input: Dict con datos adicionales para el prompt (ej: {'topic': 'X'} o {'question': 'Y'}).

    Returns:
        Tuple: (retrieved_context_str, response, retrieval_duration, llm_duration, error)
    """
    retrieved_context_str = ""
    response = ""
    retrieval_duration = 0.0
    llm_duration = 0.0
    error = None

    # 1. Recuperación
    start_time_retrieval = time.time()
    try:
        print(f"--- Retrieving context for query: '{user_query}'")
        retrieved_data = retriever_func(user_query) # Puede devolver str o List[Document]

        # Asegurarse de que el contexto sea un string para el prompt
        if isinstance(retrieved_data, list) and all(isinstance(doc, Document) for doc in retrieved_data):
             # Formato común si el retriever devuelve Documentos LangChain
            retrieved_context_str = "\n\n".join([doc.page_content for doc in retrieved_data])
            print(f"--- Retrieved {len(retrieved_data)} documents.")
        elif isinstance(retrieved_data, str):
            retrieved_context_str = retrieved_data # El retriever ya devolvió un string
            print("--- Retrieved context as a single string.")
        else:
            # Intentar convertir a string, o manejar como error si no es esperado
            print(f"--- WARNING: Unexpected retriever output type: {type(retrieved_data)}. Attempting str conversion.")
            retrieved_context_str = str(retrieved_data)

        print(f"--- Context Retrieved (first 500 chars): ---\n{retrieved_context_str[:500]}...\n-----------------------------------------")
        retrieval_duration = time.time() - start_time_retrieval

    except Exception as e:
        retrieval_duration = time.time() - start_time_retrieval
        print(f"ERROR during context retrieval for '{user_query}': {e}")
        traceback.print_exc() # Imprime el traceback completo
        retrieved_context_str = f"Error retrieving context: {e}"
        # Considerar si continuar o devolver error aquí mismo
        # return retrieved_context_str, None, retrieval_duration, 0.0, str(e)


    # 2. Generación (usando LCEL para pasar contexto y datos específicos)
    try:
        # *** INICIO DE LA CORRECCIÓN ***
        # Prepara los argumentos para assign. Cada valor debe ser un callable.
        # Usamos un argumento por defecto en el lambda interno para capturar
        # correctamente el valor de 'value' en cada iteración.
        assign_args = {
            "context": lambda x: retrieved_context_str, # Pasa el contexto recuperado
            **{key: (lambda value_copy=value: lambda x: value_copy)()
               for key, value in task_specific_input.items()} # Pasa los valores estáticos como callables
        }
        # *** FIN DE LA CORRECCIÓN ***

        print(f"--- Generating response with LLM. Prompt inputs expected: {task_prompt_template.input_variables}. Provided via assign: {list(assign_args.keys())}")

        rag_chain = (
            RunnablePassthrough.assign(**assign_args)
            | task_prompt_template
            | llm
            | StrOutputParser()
        )

        start_time_llm = time.time()
        # Invocamos la cadena. Un diccionario vacío es suficiente como input inicial
        # ya que 'assign_args' inyecta todo lo necesario para el prompt.
        response = rag_chain.invoke({})
        llm_duration = time.time() - start_time_llm
        print(f"--- LLM Response Generated (first 500 chars): ---\n{str(response)[:500]}...\n-----------------------------------------")


    except Exception as e:
        llm_duration = time.time() - start_time_llm if 'start_time_llm' in locals() else 0.0
        error_vars = task_prompt_template.input_variables if hasattr(task_prompt_template, 'input_variables') else 'N/A'
        print(f"ERROR during RAG generation (expected prompt inputs: {error_vars}): {e}")
        traceback.print_exc() # Imprime el traceback completo
        response = None # Asegurarse de que response es None en caso de error
        error = str(e)

    return retrieved_context_str, response, retrieval_duration, llm_duration, error

print("Prompts adaptados y función RAG universal (CORREGIDA) definidos.")

Prompts adaptados y función RAG universal (CORREGIDA) definidos.


In [ ]:
%pip install -q gradio

In [ ]:
import gradio as gr
import time

# --- 1. Separa tu lógica de RAG en una función limpia ---
# Esto hace que el código sea mucho más fácil de leer. Esta función
# NO debe saber nada sobre Gradio o historiales de chat.
def obtener_respuesta_rag(pregunta, modo):
    """
    Función de backend que ejecuta el pipeline de RAG y devuelve
    únicamente el string de la respuesta final.
    """
    print(f"\n--- Ejecutando RAG para: '{pregunta}' (modo={modo}) ---")

    # Elige el prompt correcto
    task_prompt_template = qa_prompt_template_cheshire if modo == "Cheshire" else qa_prompt_template_factual

    # Llama a tu función RAG universal
    contexto, respuesta, t_retrieval, t_llm, error = run_rag_based_task(
        llm=llm_gemini,
        user_query=pregunta,
        task_prompt_template=task_prompt_template,
        retriever_func=my_hybrid_rerank_retriever,
        task_specific_input={'question': pregunta}
    )

    if error:
        return f"Ups... algo se perdió en la madriguera del conejo. (Error: {error})"
    if not respuesta:
        return "Curioso... pero no encontré ninguna respuesta."

    return respuesta

if 'llm_gemini' in locals() and llm_gemini is not None:

    with gr.Blocks(theme=gr.themes.Soft(primary_hue="purple", secondary_hue="blue"), title="Chat con Cheshire") as demo:

        gr.Markdown(
            """
            <div style="text-align: center;">
                <h1>Cheshire: Conversaciones en el País de las Maravillas</h1>
                <p>Bienvenido, viajero. Has llegado a un rincón curioso. Elige a tu guía...</p>
            </div>
            """
        )

        with gr.Tabs():
            # --- PESTAÑA 1: GATO DE CHESHIRE ---
            with gr.TabItem("Gato de Cheshire 🐱"):
                with gr.Row():
                    with gr.Column(scale=3):
                        cheshire_chatbot = gr.Chatbot(
                            value=[[None, "¿Oh, un nuevo viajero? Bienvenido a este lado del espejo. Pregunta, si te atreves..."]],
                            label="Chat con Cheshire", height=550,
                            avatar_images=("/content/assets/user.png", "/content/assets/cheshire.png")
                        )
                    with gr.Column(scale=1):
                        with gr.Accordion("🔍 Ver Contexto Recuperado", open=False):
                             contexto_cheshire = gr.Markdown("El contexto recuperado aparecerá aquí...")

                with gr.Row():
                    cheshire_msg_input = gr.Textbox(label="Escribe tu pregunta para Cheshire...", scale=4, container=False)

                gr.Examples(
                    examples=["¿Qué usaban como bolas, mazos y aros en el juego de croquet de la Reina?", "¿Por qué todos aquí están locos?"],
                    inputs=cheshire_msg_input,
                    label="Ejemplos de Preguntas"
                )

                def responder_cheshire(pregunta, historial_chat):
                    historial_chat.append([pregunta, None])
                    yield historial_chat, "Recuperando un trozo del camino..."

                    contexto, respuesta, _, _, error = run_rag_based_task(
                        llm=llm_gemini, user_query=pregunta, task_prompt_template=qa_prompt_template_cheshire,
                        retriever_func=my_hybrid_rerank_retriever, task_specific_input={'question': pregunta}
                    )

                    if error: respuesta = f"Vaya... mi sonrisa se ha desvanecido. (Error: {error})"

                    historial_chat[-1][1] = ""
                    for c in respuesta:
                        historial_chat[-1][1] += c
                        time.sleep(0.02)
                        yield historial_chat, contexto

            # --- PESTAÑA 2: ASISTENTE FACTUAL ---
            with gr.TabItem("Asistente Factual 📖"):
                with gr.Row():
                    with gr.Column(scale=3):
                        factual_chatbot = gr.Chatbot(
                            value=[[None, "Modo Factual activado. ¿En qué puedo ayudarte?"]],
                            label="Chat Factual", height=550,
                            avatar_images=("/content/assets/user.png", "/content/assets/lupa.png")
                        )
                    with gr.Column(scale=1):
                        with gr.Accordion("🔍 Ver Contexto Recuperado", open=False):
                             contexto_factual = gr.Markdown("El contexto recuperado aparecerá aquí...")

                with gr.Row():
                    factual_msg_input = gr.Textbox(label="Escribe tu pregunta factual...", scale=4, container=False)

                gr.Examples(
                    examples=["¿Qué usaban como bolas, mazos y aros en el juego de croquet de la Reina?", "¿Qué animal iba corriendo con un reloj?"],
                    inputs=factual_msg_input,
                    label="Ejemplos de Preguntas"
                )

                def responder_factual(pregunta, historial_chat):
                    historial_chat.append([pregunta, None])
                    yield historial_chat, "Recuperando contexto..."

                    contexto, respuesta, _, _, error = run_rag_based_task(
                        llm=llm_gemini, user_query=pregunta, task_prompt_template=qa_prompt_template_factual,
                        retriever_func=my_hybrid_rerank_retriever, task_specific_input={'question': pregunta}
                    )

                    if error: respuesta = f"Lo siento, ocurrió un error. (Error: {error})"

                    historial_chat[-1][1] = ""
                    for c in respuesta:
                        historial_chat[-1][1] += c
                        time.sleep(0.02)
                        yield historial_chat, contexto

        # --- Conexión de Eventos para ambas pestañas ---
        cheshire_msg_input.submit(
            fn=responder_cheshire,
            inputs=[cheshire_msg_input, cheshire_chatbot],
            outputs=[cheshire_chatbot, contexto_cheshire]
        )

        factual_msg_input.submit(
            fn=responder_factual,
            inputs=[factual_msg_input, factual_chatbot],
            outputs=[factual_chatbot, contexto_factual]
        )

    demo.launch(share=True, debug=True)
else:
    print("El LLM no está inicializado.")

/tmp/ipython-input-4288668895.py:51: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  cheshire_chatbot = gr.Chatbot(
/tmp/ipython-input-4288668895.py:90: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  factual_chatbot = gr.Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c0169c2a119ab72d3e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


--- Retrieving context for query: '¿Qué usaban como bolas, mazos y aros en el juego de croquet de la Reina?'

--- (RAG Híbrido + Rerank) Buscando contexto para: '¿Qué usaban como bolas, mazos y aros en el juego de croquet de la Reina?' ---
  1. Obteniendo embedding de OpenAI...
     Embedding obtenido.
  2. Realizando búsqueda FAISS (k=100)...
     Búsqueda FAISS -> 100 candidatos.
  3. Realizando búsqueda BM25 (k=100)...
     Búsqueda BM25 -> 100 candidatos.
  4. Fusionando resultados...
  INFO DinamicWeights: Razón Principal = Nombre Propio Detectado
    - Candidatos NP: ['Qué', 'Reina']. BM25 priorizado.
  INFO DinamicWeights: Pesos Asignados -> BM25=0.65, Embedding=0.35
     Total IDs candidatos únicos: 124
     80 candidatos seleccionados para reranking.
  5. Rerankeando con 'cross-encoder/ms-marco-MiniLM-L-12-v2'...
     Reranking completado. 80 documentos rerankeados.
  6. Seleccionando chunks finales...
     Usando K Dinámico: Threshold=1.5, Min=3, Max=7
       Se seleccionaron